# **Fucntions**

In [5]:
import os
import pandas as pd
from utilities import read_text

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path+"/src")

# **Soma Identification**

This step is completed manually in Fiji.
1. Open the images in Fiji
2. Put them into a stack
3. Start marking the soma positions in each slice using the mutipoint tool
4. Once marking is complete, open ROI Manager, and then press T or click Add
5. In ROI Manager,
    1. More > Save --- save the worked ROI (just in case).
    2. Measure > Save as --- save the soma positions and the associated slice numbers with a filename Results.csv inside ~/analysis_pipeline/
6. Open Fiji_macros > get_labels.ijm
7. Click Run
8. save the results of the names and their associated slice numbers with a filename Labels.csv inside ~/analysis_pipeline/
9. Locate the associated file path in the notebook and then run

The output is a txt file for each soma in an image, saved inside ~/analysis_pipeline/vaa3d/{image_name}/###.txt

In [ ]:
# a path to the experiment
pipeline_dir = os.path.join(module_path,'src','example','analysis_pipeline')
vaa_dir = os.path.join(pipeline_dir,'vaa3d')
if not os.path.exists(vaa_dir):
    os.mkdir(vaa_dir)
    
# a path to the Results.csv
results_path = os.path.join(module_path,'src','example','analysis_pipeline','Results.csv')

# a dataframe with data from Results.csv
results_df = pd.read_csv(results_path)

# a path to the Labels.csv
labels_path = os.path.join(module_path,'src','example','analysis_pipeline','Labels.csv")

# a dataframe with data from Labels.csv
raw_txt = read_text(labels_path)
td1 = []
td2 = []

for t in raw_txt:
    tdata = t.split(' ')
    # print(tdata)
    td1.append(tdata[0])
    td2.append(tdata[1])

labels_df = pd.DataFrame({'Label':td1, 'Image':td2})
                           
prev = 0
for ind, row in results_df.iterrows():
    
    pos = row['Slice']
    if pos == prev:
        nind = 2
    x = int(row['X'])
    y = int(row['Y'])
    img_name = labels_df.loc[labels_df.Label == str(int(pos))].Image.values[0]
    output = os.path.join(vaa_dir,img_name)
    if not os.path.exists(output):
        os.mkdir(output)
    
    #print(x, y, pos, img_name)
    nind = 1
    fpath = os.path.join(output,'%s_each_%s.txt'%(img_name, nind))
    notsame = True
    while notsame:
        if os.path.exists(fpath):
            nind = nind+1
            fpath = os.path.join(output,'%s_each_%s.txt'%(img_name, nind))
        else:
            notsame = False
            
    f = open(fpath, 'w')
    f.write('%s,%s,%s,%s'%(x,y,2,nind))
    f.close()